### Reading the .csv data with pandas lib

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import daytime as dt

In [11]:
### Loading .csv file into pandas dataFrame

notif = pd.read_csv('../data/notifications.csv') # notifications

tr1 = pd.read_csv('../data/transactions_1.csv') # Transactions (3 files)
tr2 = pd.read_csv('../data/transactions_2.csv')
tr3 = pd.read_csv('../data/transactions_3.csv')
trans = pd.concat([tr1, tr2, tr3])

us = pd.read_csv('../data/users.csv') # Users


### In these steps we print some samples of the data. Get the NaN and unique values.

In [25]:
print(notif.shape)
print(notif['reason'].value_counts())
notif.sample(2)

(97704, 5)
REENGAGEMENT_ACTIVE_FUNDS            28832
PROMO                                18987
NO_INITIAL_CARD_ORDER                 9071
NO_INITIAL_CARD_USE                   9035
INVEST_IN_GOLD                        6649
LOST_CARD_ORDER                       4941
PROMO_CARD_ORDER                      4358
JOINING_ANNIVERSARY                   3334
MADE_MONEY_REQUEST_NOT_SPLIT_BILL     2851
ONBOARDING_TIPS_ACTIVATED_USERS       2423
ENGAGEMENT_SPLIT_BILL_RESTAURANT      1834
BLACK_FRIDAY                          1681
METAL_RESERVE_PLAN                     950
SILVER_ENGAGEMENT_INACTIVE_CARD        896
WELCOME_BACK                           779
BLUE_TUESDAY                           555
SILVER_ENGAGEMENT_FEES_SAVED           528
Name: reason, dtype: int64


,reason,channel,status,user_id,created_date
8015,REENGAGEMENT_ACTIVE_FUNDS,SMS,FAILED,user_11036,2018-08-10 15:46:20.009965
19492,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_2650,2018-11-13 13:15:42.332048


In [26]:
print(trans.shape)
print(trans.isna().sum())
trans.head(2)


(2181026, 12)
transaction_id                0
transactions_type             0
transactions_currency         0
amount_usd                    0
transactions_state            0
ea_cardholderpresence    933206
ea_merchant_mcc          923975
ea_merchant_city         924080
ea_merchant_country      924239
direction                     0
user_id                       0
created_date                  0
dtype: int64


,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_4368,2018-04-03 03:34:21.784487
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,FALSE,4111.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-19 06:15:59.537032


In [19]:
print(trans['transactions_type'].value_counts())

CARD_PAYMENT    1173661
TRANSFER         402126
TOPUP            309740
EXCHANGE         124800
ATM               74181
CASHBACK          65063
FEE               18786
CARD_REFUND        9209
TAX                2267
REFUND             1193
Name: transactions_type, dtype: int64


In [20]:
print(trans['transactions_state'].value_counts())

COMPLETED    1916387
DECLINED      124226
REVERTED       89330
FAILED         32566
PENDING        17153
CANCELLED       1364
Name: transactions_state, dtype: int64


In [24]:
len(trans[trans.transactions_state=='COMPLETED']) / len(trans['transactions_state'])

0.8786630695828477

In [50]:
trans1 = trans[trans.transactions_state=='COMPLETED']
trans1['created_date'] = pd.to_timedelta(trans1['created_date'])
print(trans1.info())
trans1.created_date.min()

ValueError: only leading negative signs are allowed

In [46]:
trans_grouped2 = trans1.groupby(['user_id'], as_index=True)['created_date'].agg(['min','max'])
trans_grouped2.rename(columns={'min':'first_transaction', 'max':'last_transaction'}, inplace=True)
trans_grouped2.head(2)

,first_transaction,last_transaction
user_id,,
user_0,2018-09-10 18:54:08.965855,2019-05-10 05:09:13.868855
user_1,2018-02-04 08:28:17.978146,2019-05-04 02:30:11.492146


In [49]:
trans_grouped2['transaction_period']=trans_grouped2['last_transaction']-trans_grouped2['first_transaction']
#trans_grouped2['transaction_period'].timedelta64('D')
#['transaction_period'].days
trans_grouped2.head(2)

trans_grouped2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14821 entries, user_0 to user_9999
Data columns (total 3 columns):
first_transaction     14821 non-null datetime64[ns]
last_transaction      14821 non-null datetime64[ns]
transaction_period    14821 non-null timedelta64[ns]
dtypes: datetime64[ns](2), timedelta64[ns](1)
memory usage: 463.2+ KB
